In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Data Cleaning

In [2]:
git_commits = pd.read_csv("../data/interim/git_commits.csv", encoding='utf8', engine='python')
git_commits_changes = pd.read_csv("../data/interim/git_commits_changes.csv", encoding='utf8', engine='python')
refactoring_miner = pd.read_csv("../data/interim/refactoring_miner.csv", encoding='utf8', engine='python')
szz_fault_inducing_commits = pd.read_csv("../data/interim/szz_fault_inducing_commits.csv", encoding='utf8', engine='python')

In [3]:
git_commits.info()
print("----------------------------------------------")
szz_fault_inducing_commits.info()
print("----------------------------------------------")
refactoring_miner.info()
print("----------------------------------------------")
git_commits_changes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160311 entries, 0 to 160310
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   PROJECT_ID          160311 non-null  object
 1   COMMIT_HASH         155487 non-null  object
 2   COMMIT_MESSAGE      155487 non-null  object
 3   AUTHOR              153960 non-null  object
 4   AUTHOR_DATE         153994 non-null  object
 5   AUTHOR_TIMEZONE     153994 non-null  object
 6   COMMITTER           153960 non-null  object
 7   COMMITTER_DATE      153994 non-null  object
 8   COMMITTER_TIMEZONE  153994 non-null  object
 9   BRANCHES            153994 non-null  object
 10  IN_MAIN_BRANCH      153994 non-null  object
 11  MERGE               152501 non-null  object
 12  PARENTS             152501 non-null  object
dtypes: object(13)
memory usage: 15.9+ MB
----------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52428 entries, 

## GIT_COMMITS

In [4]:
git_commits.head()

,PROJECT_ID,COMMIT_HASH,COMMIT_MESSAGE,AUTHOR,AUTHOR_DATE,AUTHOR_TIMEZONE,COMMITTER,COMMITTER_DATE,COMMITTER_TIMEZONE,BRANCHES,IN_MAIN_BRANCH,MERGE,PARENTS
0,org.apache:archiva,94fe3a7fc056638c90cbe4a6319c3cb658f395a5,create template structure git-svn-id: https:...,Brett Porter,2005-11-23 23:54:12+00:00,0,Brett Porter,2005-11-23 23:54:12+00:00,0,{'master'},True,False,[]
1,org.apache:archiva,2b6be811c1d4f5e81339616208530d486608e42b,repository manager - root POM git-svn-id: ht...,Brett Porter,2005-11-28 03:15:35+00:00,0,Brett Porter,2005-11-28 03:15:35+00:00,0,{'master'},True,False,['94fe3a7fc056638c90cbe4a6319c3cb658f395a5']
2,org.apache:archiva,af0ddbcfc1e8c2528decbd458f42e16cbcae5051,update reports git-svn-id: https://svn.apach...,Brett Porter,2005-11-28 04:50:31+00:00,0,Brett Porter,2005-11-28 04:50:31+00:00,0,{'master'},True,False,['2b6be811c1d4f5e81339616208530d486608e42b']
3,org.apache:archiva,ea640e1803a8535ebfba2ade4ea9272b240067e8,move discovery to a new component PR: MRM-9 ...,Brett Porter,2005-11-29 01:34:22+00:00,0,Brett Porter,2005-11-29 01:34:22+00:00,0,{'master'},True,False,['af0ddbcfc1e8c2528decbd458f42e16cbcae5051']
4,org.apache:archiva,4b1232d671e7c0aa44927ce2d4574d8cb751497d,add discovery module PR: MRM-9 git-svn-id: h...,Brett Porter,2005-11-29 01:43:41+00:00,0,Brett Porter,2005-11-29 01:43:41+00:00,0,{'master'},True,False,['ea640e1803a8535ebfba2ade4ea9272b240067e8']


In [5]:
len(git_commits["COMMIT_HASH"].unique())

154178

In [6]:
git_commits = git_commits[['PROJECT_ID','COMMIT_HASH','COMMIT_MESSAGE','COMMITTER_DATE','BRANCHES']]

## SZZ_FAULT_INDUCING_COMMITS

In [7]:
szz_fault_inducing_commits.head()

,PROJECT_ID,FAULT_FIXING_COMMIT_HASH,FAULT_INDUCING_COMMIT_HASH
0,org.apache:archiva,fa3688a693bc10bf53e72965027a82df0a8ad267,b7c003acb6b2c2a930443c511cb087d2776b4891
1,org.apache:archiva,d4c85abfac8d0920549f20735caf0e4028149c5c,5f9755b5ae526a83ecdd219401c97498b223b776
2,org.apache:archiva,d4c85abfac8d0920549f20735caf0e4028149c5c,9e37277c4a5d4c353f71ec5060951c554f20bab5
3,org.apache:archiva,d4c85abfac8d0920549f20735caf0e4028149c5c,28898793f51c03b1259d3a1b1f1612f450cf634c
4,org.apache:archiva,d4c85abfac8d0920549f20735caf0e4028149c5c,44dd185dade958e4c24014d84db71f870c714fcb


In [8]:
print(len(szz_fault_inducing_commits["FAULT_FIXING_COMMIT_HASH"].unique()))
print(len(szz_fault_inducing_commits["FAULT_INDUCING_COMMIT_HASH"].unique()))

7123
17326


In [9]:
szz_fault_inducing_commits = szz_fault_inducing_commits[["FAULT_FIXING_COMMIT_HASH", "FAULT_INDUCING_COMMIT_HASH"]]

## REFACTORING_MINER

In [10]:
refactoring_miner.head()

,PROJECT_ID,COMMIT_HASH,REFACTORING_TYPE,REFACTORING_DETAIL
0,org.apache:batik,adbabd6f8adad3f9df936a5d61bc2984006d0666,Move Class,Move Class\torg.apache.batik.util.XMLConstants...
1,org.apache:batik,23df647cf944b6c33ab23f0433305c153e247839,Move Class,Move Class\torg.w3c.dom.events.CustomEvent mov...
2,org.apache:batik,23df647cf944b6c33ab23f0433305c153e247839,Move Class,Move Class\torg.w3c.dom.events.KeyboardEvent m...
3,org.apache:batik,23df647cf944b6c33ab23f0433305c153e247839,Move Class,Move Class\torg.w3c.dom.events.MutationNameEve...
4,org.apache:batik,23df647cf944b6c33ab23f0433305c153e247839,Move Class,Move Class\torg.w3c.dom.events.TextEvent moved...


In [11]:
refactoring_miner[["COMMIT_HASH", "REFACTORING_TYPE"]].drop_duplicates(ignore_index=True)

,COMMIT_HASH,REFACTORING_TYPE
0,adbabd6f8adad3f9df936a5d61bc2984006d0666,Move Class
1,23df647cf944b6c33ab23f0433305c153e247839,Move Class
2,0a2576bbf0225626c1e2af1ade360ac15675fb65,Move Class
3,a0dd0a5298bb8447511759a869956ac448220db5,Move Source Folder
4,1661dbe6278577d344ebe1592308671a7c27bef2,Move Source Folder
...,...,...
10754,c631e98cad3f59565915c1cb10007ee61d960341,Move Source Folder
10755,d92ca7dcea105837e3e576f797501c19929eced5,Extract Method
10756,132cc46d07a2b92f273b37f4c163b03a001f1a5a,Rename Class
10757,d602f822d50bec166cd34c8ece170b13694268a9,Rename Class


In [12]:
refactoring_miner = refactoring_miner[["COMMIT_HASH", "REFACTORING_TYPE"]].drop_duplicates(ignore_index=True)

## GIT_COMMITS_CHANGES

In [13]:
git_commits_changes = git_commits_changes[git_commits_changes["COMMIT_HASH"].isin(refactoring_miner["COMMIT_HASH"])]
git_commits_changes = git_commits_changes[["PROJECT_ID", "COMMIT_HASH","LINES_ADDED","LINES_REMOVED"]]
git_commits_changes["LINES_ADDED"] = git_commits_changes["LINES_ADDED"].astype(int)
git_commits_changes["LINES_REMOVED"] = git_commits_changes["LINES_REMOVED"].astype(int)
git_commits_changes = git_commits_changes.groupby(['COMMIT_HASH', "PROJECT_ID"]).sum().reset_index()

In [14]:
git_commits['NUM_COMMIT'] = 0
for proj in git_commits.PROJECT_ID.unique().tolist():
    max_comm = len(git_commits[git_commits.PROJECT_ID == proj])
    git_commits.loc[git_commits.PROJECT_ID == proj, 'NUM_COMMIT'] = list(range(max_comm))

In [15]:
first_commit = {}
for proj in git_commits.PROJECT_ID.unique().tolist():
    if proj in ['org.apache:batik','org.apache:cocoon','org.apache:felix','org.apache:santuario']:
        pass
    else:
        git_commits.loc[git_commits.PROJECT_ID == proj, 'COMMITTER_DATE'] = git_commits['COMMITTER_DATE'].apply(lambda x: str(x)[:-6])
    first_commit[proj] = pd.to_datetime(git_commits[git_commits.PROJECT_ID == proj]['COMMITTER_DATE'].min()).date()
    git_commits.loc[git_commits.PROJECT_ID == proj,'COMMITTER_DATE'] = pd.to_datetime(git_commits.loc[git_commits.PROJECT_ID == proj,'COMMITTER_DATE'], dayfirst=True).dt.date
    git_commits.loc[git_commits.PROJECT_ID == proj,'COMMIT_DAY'] = git_commits.loc[git_commits.PROJECT_ID == proj,'COMMITTER_DATE'] - first_commit[proj]
git_commits['COMMIT_DAY'] = git_commits['COMMIT_DAY'].apply(lambda x: x.days)

In [16]:
git_commits_changes = git_commits_changes.merge(git_commits[['COMMIT_HASH','NUM_COMMIT', "COMMIT_DAY"]], on = 'COMMIT_HASH')
git_commits_changes = git_commits_changes.sort_values(["PROJECT_ID", "NUM_COMMIT"]).reset_index().drop("index", axis=1)
git_commits_changes['LINES_ADDED_LAST_COMMITS'] = 0
git_commits_changes['LINES_REMOVED_LAST_COMMITS'] = 0

In [17]:
days = 5
for i in range(len(git_commits_changes)):
    git_commits_changes.loc[i, "LINES_ADDED_LAST_COMMITS"] = git_commits_changes.loc[i-days:i-1, "LINES_ADDED"][git_commits_changes.PROJECT_ID == git_commits_changes.loc[i, "PROJECT_ID"]].sum()
    git_commits_changes.loc[i, "LINES_REMOVED_LAST_COMMITS"] = git_commits_changes.loc[i-days:i-1, "LINES_REMOVED"][git_commits_changes.PROJECT_ID == git_commits_changes.loc[i, "PROJECT_ID"]].sum()

In [18]:
refactor_commits = refactoring_miner[["COMMIT_HASH", "REFACTORING_TYPE"]].merge(git_commits_changes[["COMMIT_HASH", "COMMIT_DAY", "LINES_ADDED", "LINES_REMOVED", "LINES_ADDED_LAST_COMMITS", "LINES_REMOVED_LAST_COMMITS"]], on = "COMMIT_HASH")
refactor_commits["LINES_ADDED"] = refactor_commits["LINES_ADDED"].astype(int)
refactor_commits["LINES_REMOVED"] = refactor_commits["LINES_REMOVED"].astype(int)

In [19]:
refactor_commits["LABEL"] = 0
refactor_commits["LABEL"] = np.where(refactor_commits["COMMIT_HASH"].isin(szz_fault_inducing_commits["FAULT_INDUCING_COMMIT_HASH"]), 1, 0)

# Data Integration

In [20]:
refactoring_fault_inducing = refactoring_miner.merge(szz_fault_inducing_commits, left_on="COMMIT_HASH", right_on="FAULT_INDUCING_COMMIT_HASH")
refactoring_fault_inducing = refactoring_fault_inducing.drop("COMMIT_HASH", axis=1)
refactoring_fault_inducing.head()

,REFACTORING_TYPE,FAULT_FIXING_COMMIT_HASH,FAULT_INDUCING_COMMIT_HASH
0,Move Class,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,d522432b79044740831a132d8b92e7dab5477444
1,Move Class,93c12bbb6d22a165a407b6184cdb40545f86c683,d522432b79044740831a132d8b92e7dab5477444
2,Extract Class,07e0698ed6961ac535a8971e52d61e53f877f58e,02f53e4fa6ec7fa339c8c9273183669d7f5928c9
3,Extract Class,cc50f64aa8598728365f282056035cfc094e7bbd,02f53e4fa6ec7fa339c8c9273183669d7f5928c9
4,Extract Class,93c12bbb6d22a165a407b6184cdb40545f86c683,02f53e4fa6ec7fa339c8c9273183669d7f5928c9


In [21]:
fault_fixing_commits = refactoring_fault_inducing.merge(git_commits, left_on="FAULT_FIXING_COMMIT_HASH", right_on="COMMIT_HASH")
fault_fixing_commits = fault_fixing_commits.drop("COMMIT_HASH", axis=1)
fault_fixing_commits.head()

,REFACTORING_TYPE,FAULT_FIXING_COMMIT_HASH,FAULT_INDUCING_COMMIT_HASH,PROJECT_ID,COMMIT_MESSAGE,COMMITTER_DATE,BRANCHES
0,Move Class,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,d522432b79044740831a132d8b92e7dab5477444,org.apache:bcel,[BCEL-328] java.util.EmptyStackException at or...,2019-09-25 20:09:42-04:00,{'master'}
1,Rename Parameter,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,011d22895719c182ad3ddce859756154a2eb7a37,org.apache:bcel,[BCEL-328] java.util.EmptyStackException at or...,2019-09-25 20:09:42-04:00,{'master'}
2,Change Package,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,f3f9e9b02c101973da0ed1a51e5dd60b83e5bc65,org.apache:bcel,[BCEL-328] java.util.EmptyStackException at or...,2019-09-25 20:09:42-04:00,{'master'}
3,Move Class,93c12bbb6d22a165a407b6184cdb40545f86c683,d522432b79044740831a132d8b92e7dab5477444,org.apache:bcel,BCEL-287: IINC does not handle -128 properly. ...,2017-08-30 19:28:20+00:00,{'master'}
4,Extract Class,93c12bbb6d22a165a407b6184cdb40545f86c683,02f53e4fa6ec7fa339c8c9273183669d7f5928c9,org.apache:bcel,BCEL-287: IINC does not handle -128 properly. ...,2017-08-30 19:28:20+00:00,{'master'}


# Result

In [23]:
fault_fixing_commits.to_csv(r'../data/processed/fault_commits.csv', index = False)
refactor_commits.to_csv(r'../data/processed/refactor_commits.csv', index = False)

In [24]:
fault_commits = pd.read_csv("../data/processed/fault_commits.csv", encoding='utf8', engine='python')
fault_commits.head()

,REFACTORING_TYPE,FAULT_FIXING_COMMIT_HASH,FAULT_INDUCING_COMMIT_HASH,PROJECT_ID,COMMIT_MESSAGE,COMMITTER_DATE,BRANCHES,NUM_COMMIT,COMMIT_DAY
0,Move Class,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,d522432b79044740831a132d8b92e7dab5477444,org.apache:bcel,[BCEL-328] java.util.EmptyStackException at or...,2019-09-25,{'master'},1503.0,6540.0
1,Rename Parameter,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,011d22895719c182ad3ddce859756154a2eb7a37,org.apache:bcel,[BCEL-328] java.util.EmptyStackException at or...,2019-09-25,{'master'},1503.0,6540.0
2,Change Package,1d528f6cf7ea74db01a03d724f288dd97b2c3f35,f3f9e9b02c101973da0ed1a51e5dd60b83e5bc65,org.apache:bcel,[BCEL-328] java.util.EmptyStackException at or...,2019-09-25,{'master'},1503.0,6540.0
3,Move Class,93c12bbb6d22a165a407b6184cdb40545f86c683,d522432b79044740831a132d8b92e7dab5477444,org.apache:bcel,BCEL-287: IINC does not handle -128 properly. ...,2017-08-30,{'master'},1270.0,5784.0
4,Extract Class,93c12bbb6d22a165a407b6184cdb40545f86c683,02f53e4fa6ec7fa339c8c9273183669d7f5928c9,org.apache:bcel,BCEL-287: IINC does not handle -128 properly. ...,2017-08-30,{'master'},1270.0,5784.0


In [25]:
refactor_commits = pd.read_csv("../data/processed/refactor_commits.csv", encoding='utf8', engine='python')
refactor_commits.head()

,COMMIT_HASH,REFACTORING_TYPE,COMMIT_DAY,LINES_ADDED,LINES_REMOVED,LINES_ADDED_LAST_COMMITS,LINES_REMOVED_LAST_COMMITS,LABEL
0,23df647cf944b6c33ab23f0433305c153e247839,Move Class,6421,19,14,957,42,0
1,0a2576bbf0225626c1e2af1ade360ac15675fb65,Move Class,6420,21,16,939,32,0
2,a0dd0a5298bb8447511759a869956ac448220db5,Move Source Folder,6170,40,3,1101,29,0
3,1661dbe6278577d344ebe1592308671a7c27bef2,Move Source Folder,6169,108,1,996,31,0
4,49d15d8f0c66f1b41c1aab24430f129388826e89,Move Source Folder,5625,748,22,294,9,0


# Data Construction

In [26]:
string = ""

for i in range(len(fault_fixing_commits)):
    git_message = fault_fixing_commits.loc[i, 'COMMIT_MESSAGE'].split("\n") # Break string into lines
    git_message = [s for s in git_message if '//' not in s]  # Remove lines with a link
    git_message = "\n".join(git_message).strip() # Join again the different lines and remove empty ones

    # Append commit messages with a point, if they don't have it yet
    if git_message != "":
        if git_message[-1] == ".":
            string += git_message
        else:
            string += git_message + ". "

In [27]:
string

"[BCEL-328] java.util.EmptyStackException at org.apache.bcel.classfile.DescendingVisitor.visitModule (DescendingVisitor.java:592).[BCEL-328] java.util.EmptyStackException at org.apache.bcel.classfile.DescendingVisitor.visitModule (DescendingVisitor.java:592).[BCEL-328] java.util.EmptyStackException at org.apache.bcel.classfile.DescendingVisitor.visitModule (DescendingVisitor.java:592).[BCEL-338] org.apache.bcel.generic.InstructionFactory.createInvoke() populates its internal ConstantPoolGen on bad input  then throws exception.[BCEL-338] org.apache.bcel.generic.InstructionFactory.createInvoke() populates its internal ConstantPoolGen on bad input  then throws exception.[BCEL-338] org.apache.bcel.generic.InstructionFactory.createInvoke() populates its internal ConstantPoolGen on bad input  then throws exception.[BCEL-338] org.apache.bcel.generic.InstructionFactory.createInvoke() populates its internal ConstantPoolGen on bad input  then throws exception.[BCEL-339] Throw IllegalArgumentExce